In [ ]:
!conda install -y pandas numpy

### Preparing data for multi-label image classification

Each original chest x-ray image is 1024×1024 in size. We used 224×224 input resolution to obtain initial results, and then we obtained results on the full resolution. The input dataset is split into training/validation and testing without patient overlap between these sets and the 14 diseases are modeled as 14 binary outputs of a multi-label classifier. Each disease output indicates the presence/absence of the disease and “no disease” is represented as zeros for all labels. We first split the data into three sets: training, validation, and testing, based on patient IDs.

In [16]:
import pandas as pd
import numpy as np
import urllib.request
import boto3

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()
bucket='analytics-serverless-west'
prefix = 'sagemaker/x-ray'

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

        
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def download_from_s3(bucket, key, local_file_name):
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).download_file(key, local_file_name)

trainper = 0.7
valper = 0.1
file_name = 'Data_Entry_2017.csv'

download_from_s3(bucket, prefix+'/'+file_name, file_name)

a = pd.read_csv(file_name)
patient_ids = a['Patient ID']
uniq_pids = np.unique(patient_ids)
np.random.shuffle(uniq_pids)
total_ids = len(uniq_pids)
#total_ids = 30000 #pick a sample size you want to use to train and validate the model

In [4]:
a.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


### Make Train, Validation, and Testing datasets from the full image list

In [6]:
trainset = int(trainper*total_ids)
valset = trainset+int(valper*total_ids)
testset = trainset+valset

train = uniq_pids[:trainset]
val = uniq_pids[trainset+1:valset]
test = uniq_pids[valset+1:]
print('Number of patient ids: training: %d, validation: %d, testing: %d'%(len(train), len(val), len(test)))

traindata = a.loc[a['Patient ID'].isin(train)]
valdata = a.loc[a['Patient ID'].isin(val)]
testdata = a.loc[a['Patient ID'].isin(test)]

traindata.to_csv('traindata.csv', sep=',', header=False, index=False)
valdata.to_csv('valdata.csv', sep=',', header=False, index=False)
testdata.to_csv('testdata.csv', sep=',', header=False, index=False)

Number of patient ids: training: 21563, validation: 3079, testing: 6161


Then, we create the ‘[lst](https://mxnet.incubator.apache.org/faq/recordio.html)’ files for multi-label classification where each disease is mapped to a set of binary labels.

In [7]:
import csv

def gen_set(csvfile, outputfile):
    disease_list = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', \
                   'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', \
                   'Hernia']
    alldiseases = {disease:i for i,disease in enumerate(disease_list)}
    with open(outputfile, 'w') as fp:
        with open(csvfile, 'r') as cfile:
            line = csv.reader(cfile, delimiter=',')
            index = 0
            for element in line:
                # the first column is the image filename, while the second
                # column has the list of diseases separated by |
                diseases = element[1].split('|')
                fp.write('%d\t'%index)
                for d in alldiseases:
                    if d in diseases:
                        fp.write('%d\t'%1)
                    else:
                        fp.write('%d\t'%0)
                fp.write('images/%s\n' % element[0])
                index += 1
                 
gen_set('traindata.csv', 'chestxraytrain.lst')
gen_set('valdata.csv', 'chestxrayval.lst')
gen_set('testdata.csv', 'chestxraytest.lst')      

# Snippet of the annotated training data

In [8]:
t_csv = pd.read_csv('traindata.csv')
v_csv = pd.read_csv('valdata.csv')

t_csv.head()

,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.14300000000000002,0.14300000000000002.1,Unnamed: 11
0,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
1,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
2,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
3,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
4,00000003_001.png,Hernia,1,3,74,F,PA,2500,2048,0.168,0.168,NaN


# Snippet of the annotated validation data

In [114]:
v_csv.head()

,00000008_000.png,Cardiomegaly,0,8,69,F,PA,2048,2500,0.171,0.171.1,Unnamed: 11
0,00000008_001.png,No Finding,1,8,70,F,PA,2048,2500,0.171,0.171,NaN
1,00000008_002.png,Nodule,2,8,73,F,PA,2048,2500,0.168,0.168,NaN
2,00000014_000.png,No Finding,0,14,61,F,PA,2048,2500,0.171,0.171,NaN
3,00000040_000.png,No Finding,0,40,67,M,PA,2500,2048,0.168,0.168,NaN
4,00000040_001.png,Emphysema,1,40,67,M,AP,2500,2048,0.168,0.168,NaN


Then we create recordio files using [im2rec.py](https://github.com/apache/incubator-mxnet/blob/master/tools/im2rec.py) for training and validation. We pass the option pack_label so that the recordio file is created as a multi-label file. For more details, please refer to the Amazon SageMaker multi-label image classification notebooks.

In [115]:
download('https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py')

### Only run the below cell once. This is an expensive operation

In [ ]:
!aws s3 sync s3://analytics-serverless-west/sagemaker/x-ray/images/ images/

In [116]:
!head -10 chestxraytest.lst

0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000002_000.png
1	0	0	0	0	0	1	0	0	0	0	0	0	0	0	images/00000009_000.png
2	0	0	1	0	0	0	0	0	0	0	0	0	0	0	images/00000010_000.png
3	0	0	0	0	0	0	0	1	0	0	0	0	0	0	images/00000011_000.png
4	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000011_001.png
5	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000011_002.png
6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000011_003.png
7	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000011_004.png
8	0	0	1	0	0	0	0	0	0	0	0	0	0	0	images/00000011_005.png
9	1	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000011_006.png


In [117]:
!python im2rec.py --pack-label chestxraytrain.lst .
!python im2rec.py --pack-label chestxrayval.lst .

Creating .rec file from /home/ec2-user/SageMaker/chestxraytrain.lst in /home/ec2-user/SageMaker
multiprocessing not available, fall back to single threaded encoding
time: 0.0586705207824707  count: 0
libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
time: 53.599899768829346  count: 1000
time: 53.78526711463928  count: 2000
time: 54.013200521469116  count: 3000
time: 53.6976683139801  count: 4000
time: 53.34519624710083  count: 5000
time: 54.07750463485718  count: 6000
time: 53.62135601043701  count: 7000
libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
time: 53.41580605506897  count: 8000
time: 54.20130252838135  count: 9000
time: 53.700379610061646  count: 10000
time: 54.05881881713867  count: 11000
time: 54.3556752204895  count: 12000
time: 54.17163586616516  count: 13000
time: 53.96117687225342  count: 14000
time: 54.45259714126587  count: 15000
time: 53.820563554763794  count: 16000
time: 54.

### Upload to S3

In [118]:
import boto3
import os

bucket = 'analytics-serverless-west'
prefix = 'sagemaker/x-ray'

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', 'chestxraytrain.rec')).upload_file('chestxraytrain.rec')
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', 'chestxrayval.rec')).upload_file('chestxrayval.rec')
s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
print('uploaded validation data location: {}'.format(s3_validation_data))

# boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', 'chestxraytrain.lst')).upload_file('chestxraytrain.lst')
# s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
# print('uploaded training data location: {}'.format(s3_train_data))

# boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', 'chestxrayval.lst')).upload_file('chestxrayval.lst')
# s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
# print('uploaded validation data location: {}'.format(s3_validation_data))


uploaded training data location: s3://analytics-serverless-west/sagemaker/x-ray/train/recordio-pb-data
uploaded validation data location: s3://analytics-serverless-west/sagemaker/x-ray/validation/recordio-pb-data


### Image classification results on the chest x-ray dataset

We used the ResNet-50 model and first trained the network with 224×224 input image size. We used data augmentation techniques such as random cropping, and image transformations. Even though a chest x-ray image is different from ImageNet images, using a pre-trained model trained on the ImageNet dataset helps in achieving better classification accuracy. Hence, we used the use_pretrained_model hyperparameter in the Amazon SageMaker image classification algorithm to train the network. Since this is a multi-label classification, we set the multi_label parameter to 1. We resized the chest x-ray images to 256 before training so that the network can crop 224×224 regions from the input image.

The following code snippet shows how it can be done using the [Amazon SageMaker Estimator interface](https://sagemaker.readthedocs.io/en/latest/estimators.html) and the image classification algorithm.

In [9]:
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
print(s3train)

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)


s3://analytics-serverless-west/sagemaker/x-ray/train/


In [10]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role,
                        train_instance_count=1,
                        train_instance_type='ml.p3.16xlarge',
                        train_volume_size = 50, train_max_run = 360000,
                        input_mode= 'File', output_path=s3_output_location,
                        sagemaker_session=sess)
multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
                                        image_shape = "3,224,224", num_classes=14,
                                        mini_batch_size=256, 
                                        resize=256,  epochs=100, 
                                        learning_rate=0.0005, optimizer='adam', 
                                        num_training_samples=80000,
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float32', multi_label = 1)
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)


INFO:sagemaker:Creating training-job with name: image-classification-2018-11-13-19-36-44-842


2018-11-13 19:36:44 Starting - Starting the training job...
2018-11-13 19:36:47 Starting - Launching requested ML instances............
2018-11-13 19:38:50 Starting - Preparing the instances for training.........
2018-11-13 19:40:20 Downloading - Downloading input data
2018-11-13 19:40:20 Stopping - Stopping the training job
2018-11-13 19:40:20 Stopped - Training job stopped
..
Billable seconds: 1


### Training with weighted loss

An additional feature introduced in image classification is the use of weighted loss to handle class imbalance. Typically, when training with a multi-label dataset, there might be imbalance between classes. This imbalance can lead to a network leaning towards learning one class over another. To avoid that, the Amazon SageMaker image classification algorithm uses the use_weighted_loss hyperparameter to balance the samples. When this parameter is set to 1, a weight value is calculated for each label based on the number of samples of that label in the training set. First, the number of samples in each class is calculated from the training set and the weight for loss update is set to N/N_l for that class where N is the total number of samples in the training set and N_l is the total number of samples for class l in the training set. This will weigh the loss calculated for gradient update differently for each class based on their weight thereby enabling balanced training. The average AUC increased to 0.814 when trained using the weighted loss feature enabled while still using 224×224 input resolution.

In [ ]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role,
                        train_instance_count=1,
                        train_instance_type='ml.p3.16xlarge',
                        train_volume_size = 50, train_max_run = 360000,
                        input_mode= 'File', output_path=s3_output_location,
                        sagemaker_session=sess)
# multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
#                                         image_shape = "3,224,224", num_classes=14,
#                                         mini_batch_size=256, 
#                                         resize=256,  epochs=100, 
#                                         learning_rate=0.0005, optimizer='adam', 
#                                         num_training_samples=80000,
#                                         augmentation_type = 'crop_color_transform',
#                                         precision_dtype='float32', multi_label = 1)
multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
                                        image_shape = "3,224,224", num_classes=14,
                                        mini_batch_size=256, resize=256,  epochs=100, 
                                        learning_rate=0.0005, optimizer='adam', 
                                        num_training_samples=80000, use_weighted_loss=1,
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float32', multi_label = 1)
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)


### Training with mixed-precision

The Amazon SageMaker image classification algorithm now supports training in mixed-precision mode. This is controlled by the hyperparameter, precision_dtype, which can be set to ‘float32’ (default) or ‘float16’. In mixed-precision mode, the network computes the backward and forward pass in low-precision (float16) while maintaining the master weights in high-precision (float32). This enables the training to be faster while maintaining similar accuracy. By using the mixed-precision mode, the training time was reduced by 33 percent while obtaining the overall AUC of 0.821, which is similar to the one obtained with float32 training. The training time reduction was even greater when training using two instances for the high-resolution input (see the following section) and increased to 47 percent.

In [121]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role,
                        train_instance_count=1,
                        train_instance_type='ml.p3.16xlarge',
                        train_volume_size = 50, train_max_run = 360000,
                        input_mode= 'File', output_path=s3_output_location,
                        sagemaker_session=sess)
# multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
#                                         image_shape = "3,224,224", num_classes=14,
#                                         mini_batch_size=256, 
#                                         resize=256,  epochs=100, 
#                                         learning_rate=0.0005, optimizer='adam', 
#                                         num_training_samples=80000,
#                                         augmentation_type = 'crop_color_transform',
#                                         precision_dtype='float32', multi_label = 1)
multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
                                        image_shape = "3,224,224", num_classes=14,
                                        mini_batch_size=256, resize=256, epochs=100, 
                                        learning_rate=0.0005, optimizer='adam', 
                                        num_training_samples=80000, use_weighted_loss=1, 
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float16', multi_label = 1)

train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating training-job with name: image-classification-2018-11-04-01-02-01-981


2018-11-04 01:02:02 Starting - Starting the training job...
2018-11-04 01:02:05 Starting - Launching requested ML instances.........
2018-11-04 01:03:37 Starting - Preparing the instances for training......
2018-11-04 01:04:45 Downloading - Downloading input data...
2018-11-04 01:05:12 Failed - Training job failed
..

ValueError: Error training image-classification-2018-11-04-01-02-01-981: Failed Reason: ClientError: Data download failed:S3 key: s3://analytics-serverless-west/validation/ matched no files on s3

### Training with high-resolution input

We then used the original input resolution by setting the image_shape parameter to 896×896. We used the use_weighted_loss feature and float32 precision for this training. We used this resolution because it allows the network to sample a 896×896 region from the 1024×1024 during data augmentation. Since the high resolution will use more memory, typically batch_size is reduced to train the network. However, because Amazon SageMaker image classification supports distributed training, we were able to maintain the batch_size by running the training across multiple instances. This is done by setting the [instance_count parameter](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html) in the Amazon SageMaker training to 2. The average AUC for this resolution increased to 0.830, particularly for classes such as nodule, which can benefit from high-resolution input. When we trained with mixed_precision set to 1, the average AUC was 0.825. The training was done using the same code as before but setting the train_instance_count = 2, image_shape=”3,896,896” and not setting the resize parameter.



In [ ]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role, train_instance_count=2,
                                                train_instance_type='ml.p3.16xlarge',
                                                train_volume_size = 50, train_max_run = 360000,
                                                input_mode= 'File', output_path=s3_output_location,
                                                sagemaker_session=sess)

multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1, 
                                        image_shape = "3,896,896", num_classes=14,
                                        mini_batch_size=64, epochs=100, 
                                        learning_rate=0.00025, optimizer='adam', 
                                        num_training_samples=80000, use_weighted_loss=1, 
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float32', multi_label = 1)

train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)

| Label | 224×224 | 224×224 with class balancing | 224×224 with mixed precision | 896×896 |
|-----------| :--------------------------: | :--------------------------: | :-------: | ---------: |
|Atelectasis | 0.772 |0.802 | 0.799 | 0.800 |
|Cardiomegaly | 0.859 | 0.899 | 0.906 | 0.884 |
|Effusion | 0.830 | 0.873 | 0.873 | 0.873 |
|Infiltration | 0.626 | 0.693 | 0.691 | 0.698 |
|Mass | 0.791 | 0.839 | 0.834 | 0.821 |
|Nodule | 0.716 | 0.743 | 0.751 | 0.817 |
|Pneumonia | 0.645 | 0.710 | 0.713 | 0.739 |
|Pneumothorax | 0.778 | 0.836 | 0.862 | 0.878 |
|Consolidation | 0.695 | 0.791 | 0.789 | 0.785 |
|Edema | 0.799 | 0.849 | 0.863 | 0.879| 
|Emphysema | 0.850 | 0.889 | 0.909 | 0.933 |
|Fibrosis | 0.764 | 0.791 | 0.811 | 0.822 |
|Pleural Thickening | 0.726 | 0.758 | 0.761 | 0.785 |
|Hernia | 0.903 | 0.929 | 0.940 | 0.911 |
|Average AUC | 0.768 | 0.814 | 0.821 | 0.830 |


### Deploy the model

***

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the topic mixture representing a given document.

Image-classification only supports encoded .jpg and .png image formats as inference input for now. The output is the probability values for all classes encoded in JSON format, or in JSON Lines format for batch transform.

This section involves several steps,

1. [Create Model](#CreateModel) - Create model for the training output
1. [Batch Transform](#BatchTransform) - Create a transform job to perform batch inference.
1. [Host the model for realtime inference](#HostTheModel) - Create an inference endpoint and perform realtime inference.

## Create Model

We now create a SageMaker Model from the training output. Using the model we can create a Batch Transform Job or an Endpoint Configuration.

In [9]:
%%time
import boto3
from time import gmtime, strftime

sage = boto3.Session().client(service_name='sagemaker') 

# get the name of the training job completed below for this variable
job_name="image-classification-2018-11-03-07-10-36-441"
model_name = "x-ray-image-classification-model"
print(model_name)
info = sage.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

x-ray-image-classification-model
s3://analytics-serverless-west/sagemaker/x-ray/output/image-classification-2018-11-03-07-10-36-441/output/model.tar.gz
arn:aws:sagemaker:us-west-2:649037252677:model/x-ray-image-classification-model
CPU times: user 44.8 ms, sys: 0 ns, total: 44.8 ms
Wall time: 407 ms


### Realtime inference

We now host the model with an endpoint and perform realtime inference.

This section involves several steps,
1. [Create endpoint configuration](#CreateEndpointConfiguration) - Create a configuration defining an endpoint.
1. [Create endpoint](#CreateEndpoint) - Use the configuration to create an inference endpoint.
1. [Perform inference](#PerformInference) - Perform inference on some input data using the endpoint.
1. [Clean up](#CleanUp) - Delete the endpoint and model

#### Create Endpoint Configuration
At launch, we will support configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way.

In addition, the endpoint configuration describes the instance type required for model deployment, and at launch will describe the autoscaling configuration.

In [12]:
import time
from time import gmtime, strftime

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = model_name + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration name: x-ray-image-classification-model-epc--2018-11-03-16-23-12
Endpoint configuration arn:  arn:aws:sagemaker:us-west-2:649037252677:endpoint-config/x-ray-image-classification-model-epc--2018-11-03-16-23-12


#### Create Endpoint
Lastly, the customer creates the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [16]:
%%time
import time

sage = boto3.client('sagemaker')
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = model_name + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sage.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: x-ray-image-classification-model-ep--2018-11-03-16-26-35
EndpointArn = arn:aws:sagemaker:us-west-2:649037252677:endpoint/x-ray-image-classification-model-ep--2018-11-03-16-26-35
CPU times: user 29.7 ms, sys: 0 ns, total: 29.7 ms
Wall time: 373 ms


In [17]:
# get the status of the endpoint
response = sage.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))


# wait until the status has changed
sage.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)


# print the status of the endpoint
endpoint_response = sage.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

EndpointStatus = Creating
Endpoint creation ended with EndpointStatus = InService


If you see the message,

> `Endpoint creation ended with EndpointStatus = InService`

then congratulations! You now have a functioning inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.

We will finally create a runtime object from which we can invoke the endpoint.

#### Perform Inference
Finally, the customer can now validate the model for use. They can obtain the endpoint from the client library using the result from previous operations, and generate classifications from the trained model using that endpoint.


In [3]:
import boto3
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

In [4]:
!head -10 chestxraytest.lst

0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000005_000.png
1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000005_001.png
2	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000005_002.png
3	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000005_003.png
4	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000005_004.png
5	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000005_005.png
6	0	0	1	0	0	0	0	0	0	0	0	0	0	0	images/00000005_006.png
7	0	0	1	0	0	0	0	1	0	0	0	0	0	0	images/00000005_007.png
8	0	0	0	0	0	0	0	0	0	0	0	0	0	0	images/00000007_000.png
9	0	0	1	0	0	0	0	0	0	0	0	0	0	0	images/00000010_000.png


In [85]:
files_list = ! aws s3 ls s3://analytics-serverless-west/sagemaker/x-ray/images/ | cut -d " " -f 8

In [113]:
from IPython.display import Image
import json
import numpy as np
import csv

In [194]:
results=[]
detection=[]
detections=[]
totalprocessed=0
perfect=0
perfectnl=0
acwfp=0
scwnfp=0
ncwnfp=0
scwfp=0
ncwfp=0

with open('Data_Entry_2017.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    csv_reader.__next__()
    for row in csv_reader:
        ! aws s3 cp s3://analytics-serverless-west/sagemaker/x-ray/images/{row[0]} . --quiet
        with open(row[0], 'rb') as f:
            payload = f.read()
            payload = bytearray(payload)
        endpoint_name = 'x-ray-image-classification-model-ep--2018-11-03-16-26-35'
        response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType='application/x-image', 
                                           Body=payload)
        result = response['Body'].read()
        # result will be in json format and convert it to ndarray
        result = json.loads(result)
        # the result will output the probabilities for all classes
        # find the class with maximum probability and print the class index
        index = np.argmax(result)
        disease_list = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', \
                           'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', \
                           'Hernia']

        detection=[]



        
        # Original thresholds
        for idx, val in enumerate(result):
           switcher1 = {
               "Atelectasis": 0.17162750661373138,
               "Consolidation": 0.13634157180786133,
               "Infiltration": 0.17990991473197937,
               "Pneumothorax": 0.08858224004507065,
               "Edema": 0.066930390894413,
               "Emphysema": 0.03022913448512554,
               "Fibrosis": 0.02981789968907833,
               "Effusion": 0.24083951115608215,
               "Pneumonia": 0.012233437038958073,
               "Pleural_Thickening": 0.08263493329286575,
               "Cardiomegaly": 0.13562433421611786,
               "Nodule": 0.047457192093133926,
               "Mass": 0.034243155270814896,
               "Hernia": 0.011998575180768967,
           }

        
#        Original thresholds * 4
#        for idx, val in enumerate(result):
#            switcher1 = {
#                "Atelectasis": 0.686510026454924,
#                "Consolidation": 0.545366287231444,
#                "Infiltration": 0.719639658927916,
#                "Pneumothorax": 0.354328960180282,
#                "Edema": 0.267721563577652,
#                "Emphysema": 0.120916537940502,
#                "Fibrosis": 0.119271598756313,
#                "Effusion": 0.963358044624328,
#                "Pneumonia": 0.048933748155832,
#                "Pleural_Thickening": 0.330539733171463,
#                "Cardiomegaly": 0.542497336864468,
#                "Nodule": 0.189828768372536,
#                "Mass": 0.136972621083259,
#                "Hernia": 0.047994300723076,
#            }


        if (result[idx] >= (switcher1.get(disease_list[idx], 1))):
#            print(disease_list[idx]) 
#            print(switcher.get(disease_list[idx], 1)*2)
#            print(switcher.get(disease_list[idx], 1))
            detection.append(disease_list[idx])
        correctlabelcount=0
        for label in row[1].split("|"):
            if label in detection:
                correctlabelcount=correctlabelcount+1
        printed=0
        if (correctlabelcount == len(row[1].split("|")) and len(row[1].split("|")) == len(detection)):
            results.append("Perfect Match")
 #           print(row[0], " Perfect Match")
 #           print("Detected: ", detection)
 #           print("Actual:", row[1].split("|"))
            printed=1
            perfect=perfect+1
            totalprocessed=totalprocessed+1
        if (len(detection) == 0 and row[1] == "No Finding"):
            results.append("Perfect Match, No Findings")
 #           print(row[0], " Perfect Match, No Findings")
 #           print("Detected: ", detection)
 #           print("Actual:", row[1].split("|"))
            printed=1
            perfectnl=perfectnl+1
            totalprocessed=totalprocessed+1
        if (correctlabelcount == len(row[1].split("|")) and len(row[1].split("|")) < len(detection)):
            results.append("Detected all correct labels, with false positives")
 #           print(row[0], " Detected all correct labels, with false positives")
 #           print("Detected: ", detection)
 #           print("Actual:", row[1].split("|"))
            printed=1
            acwfp=acwfp+1
            totalprocessed=totalprocessed+1
        if (correctlabelcount < len(row[1].split("|")) and correctlabelcount != 0 and correctlabelcount == len(detection)):
            results.append("Detected some correct labels, with no false positives")
 #           print(row[0], " Detected some correct labels, with no false positives")
 #           print("Detected: ", detection)
 #           print("Actual:", row[1].split("|"))
            printed=1
            scwnfp=scwnfp+1
            totalprocessed=totalprocessed+1
        if (len(detection) == 0 and row[1] != "No Finding"):
            results.append("Detected no correct labels, with no false positives")
#            print(row[0], " Detected no correct labels, with no false positives")
#            print("Detected: ", detection)
#            print("Actual:", row[1].split("|"))
            printed=1
            ncwnfp=ncwnfp+1
            totalprocessed=totalprocessed+1
        if (correctlabelcount < len(row[1].split("|")) and correctlabelcount != 0 and correctlabelcount < len(detection)):
            results.append("Detected some correct labels, with false positives")
#            print(row[0], " Detected some correct labels, with false positives")
#            print("Detected: ", detection)
#            print("Actual:", row[1].split("|"))
            printed=1
            scwfp=scwfp+1
            totalprocessed=totalprocessed+1
        if (correctlabelcount == 0 and len(detection) != 0 and row[1] != "No Finding"):
            results.append("Detected no correct labels, with false positives")
#            print(row[0], " Detected no correct labels, with false positives")
#            print("Detected: ", detection)
#            print("Actual:", row[1].split("|"))
            printed=1
            ncwfp=ncwfp+1
            totalprocessed=totalprocessed+1
        if (len(detection) > 0 and row[1] == "No Finding"):
            results.append("Detected no correct labels, with false positives")
#            print(row[0], " Detected no correct labels, with false positives")
#            print("Detected: ", detection)
#            print("Actual:", row[1].split("|"))
            printed=1
            ncwfp=ncwfp+1
            totalprocessed=totalprocessed+1
        if printed == 0:
#            print(row[0], " Didn't match any scenarios")
#            print("Detected: ", detection)
#            print("Actual:", row[1].split("|"))
            totalprocessed=totalprocessed+1
        if totalprocessed == 10000:
            break

        ! rm {row[0]}
print("Perfect Matches: ", perfect)
print("Perfect Match, no labels: ", perfectnl)
print("All labels correct with false positives: ", acwfp)
print("Some labels correct with no false positives: ", scwnfp)
print("No labels correct with no false positives: ", ncwnfp)
print("Some labels correct with false positives: ", scwfp)
print("No labels correct with false positives: ", ncwfp)
print("score: ", (perfect*1)+(acwfp*0.5)+(scwnfp*0.25)+(ncwnfp*0.25)+(scwfp*0.25))
print("total processed: ", totalprocessed)
print("weighted score:", ((perfect*1)+(perfectnl*1)+(acwfp*0.5)+(scwnfp*0.25)+(ncwnfp*0.25)+(scwfp*0.25))/totalprocessed)

Perfect Matches:  12
Perfect Match, no labels:  5442
All labels correct with false positives:  0
Some labels correct with no false positives:  7
No labels correct with no false positives:  3957
Some labels correct with false positives:  0
No labels correct with false positives:  582
score:  1003.0
total processed:  10000
weighted score: 0.6445


In [168]:
import csv

with open('Data_Entry_2017.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    csv_reader.__next__()
    csv_reader.__next__()
    print(csv_reader.__next__()[1].split("|"))
    print(csv_reader.__next__()[0])
#    for row in csv_reader:
#            print(f'\t{row[0]}')


['Cardiomegaly', 'Emphysema']
00000001_002.png


In [138]:
print("score: ", score)
print("total processed: ", totalprocessed)
print("weighted score:", score/totalprocessed)

score:  10.75
total processed:  50
weighted score: 0.215


In [ ]:
        # Original thresholds
        for idx, val in enumerate(result):
           switcher1 = {
               "Atelectasis": 0.17162750661373138,
               "Consolidation": 0.13634157180786133,
               "Infiltration": 0.17990991473197937,
               "Pneumothorax": 0.08858224004507065,
               "Edema": 0.066930390894413,
               "Emphysema": 0.03022913448512554,
               "Fibrosis": 0.02981789968907833,
               "Effusion": 0.24083951115608215,
               "Pneumonia": 0.012233437038958073,
               "Pleural_Thickening": 0.08263493329286575,
               "Cardiomegaly": 0.13562433421611786,
               "Nodule": 0.047457192093133926,
               "Mass": 0.034243155270814896,
               "Hernia": 0.011998575180768967,
           }
            
            
            

            
        # 20% false positive rate in each class thresholds
        for idx, val in enumerate(result):
           switcher1 = {
               "Atelectasis": 0.778095006942749,
               "Consolidation": 1.2920989692211151,
               "Infiltration": 1.4605742394924164,
               "Pneumothorax": 0.6257116198539734,
               "Edema": 0.2713622748851776,
               "Emphysema": 0.7842891812324524,
               "Fibrosis": 0.39154180884361267,
               "Effusion": 0.6497185230255127,
               "Pneumonia": 1.0556025952100754,
               "Pleural_Thickening": 1.3903022706508636,
               "Cardiomegaly": 0.7538480162620544,
               "Nodule": 1.3643558323383331,
               "Mass": 1.2591769099235535,
               "Hernia": 0.05428028479218483,
           }        
            
            
            